In [1]:
import numpy as np
from tqdm import tqdm

f = open("/mnt/c/Users/cymle/Desktop/data/bill_detail.csv", 'r')
f.readline()
# uid,time,bank_id,last_bill_amount,last_pay_amount,credit_amount,cur_bill_bal,cur_bill_minerapay,cons_num,cur_bill_amount,adjust_amount,circle_interest,available_amount,"precash_amount,",repay_state
# + 1234

f1 = open("/mnt/c/Users/cymle/Desktop/data/user_info.csv", 'r')
f1.readline()
uid_labels = {}
for x in f1.readlines():
    li = x.strip().split(',')
    uid_labels[li[0]] = li[0] + ','

outputs = {}
users = []

def do_helper(f_list, func_obj="sum"):
    a = np.array(f_list[0:11]+f_list[12:])
    if func_obj == 'sum':
        res = np.sum(a, axis=0)
    elif func_obj == "mean":
        res = np.mean(a, axis=0)
    elif func_obj == "max":
        res = np.max(a, axis=0)
    elif func_obj == "min":
        res = np.min(a, axis=0)
    elif func_obj == "std":
        res = np.std(a, axis=0)
    elif func_obj == "25":
        res = np.percentile(a, 25, axis=0)
    elif func_obj == "50":
        res = np.percentile(a, 50, axis=0)
    elif func_obj == "75":
        res = np.percentile(a, 75, axis=0)
    res = res.tolist()
    assert len(res) == 16
    return res[0:11]+res[12:]

def map_list2str(li):
    msg = ""
    if not isinstance(li, list):
        msg = str(li) + ','
        return msg
    for x in li:
        msg += str(x) + ','
    return msg

In [2]:
### section 1
print("section 1")
all_data = []
for line in tqdm(f.readlines()):
    li = line.strip().split(',')
    features = []
    for x in li[:3]:
        features.append(x)
    for x in li[3:]:
        features.append(float(x))
    features.append(features[3] - features[4])
    features.append(features[9] - features[3])
    features.append(features[3] - features[4] + features[9] - features[10] + features[11])
    features.append(features[5] - features[12])
    assert len(features)==19
    all_data.append(features)

section 1


100%|██████████| 2338118/2338118 [00:18<00:00, 126150.23it/s]


In [3]:
### section 2
print("section 2")
uid_dict = {}
for item in all_data:
    [uid, ctime, bid] = item[0:3]
    if uid not in uid_dict:
        users.append(uid)
        outputs[uid] = ""
        uid_dict[uid] = {
            "bank": set(),
            "cnt": 0,
            "f_list": [],
        }
    uid_dict[uid]["bank"].add(bid)
    uid_dict[uid]["cnt"]+=1
    uid_dict[uid]['f_list'].append(item[3:])

uid_res = {}    
for uid, v in tqdm(uid_dict.items()):
    f_list = v['f_list']
    uid_res[uid] = {
        "bank_size": len(v["bank"]),
        "cnt": v["cnt"],
        "sum_features": do_helper(v["f_list"], "sum"),
        "mean_features": do_helper(v["f_list"], "mean"),
        "max_features": do_helper(v["f_list"], "max"),
        "min_features": do_helper(v["f_list"], "min"),
        "std_features": do_helper(v["f_list"], "std"),
        "25median_features": do_helper(v["f_list"], "25"),
        "50median_features": do_helper(v["f_list"], "50"),
        "75median_features": do_helper(v["f_list"], "75"),
    }

for uid in users:
    if uid in uid_res:
        for key in sorted(uid_res[uid].keys()):
            outputs[uid] += map_list2str(uid_res[uid][key])
    else:
        assert "uid:{} miss in uid_res".format(uid)

section 2


100%|██████████| 53174/53174 [00:33<00:00, 1582.26it/s]


In [4]:
### section 3
print("section 3")
uid_last_time = {}
uid_bid_dict = {}
banks = set()


for item in all_data:
    [uid, ctime, bid] = item[0:3]
    bid = int(bid)
    if bid < 10:
        bid = "0~9"
    elif bid < 20:
        bid = "10~19"
    else:
        bid = "other"
    ctime = int(ctime)
    if uid not in uid_bid_dict:
        uid_bid_dict[uid] = {}
    if bid not in uid_bid_dict[uid]:
        uid_bid_dict[uid][bid] = {
            "cnt": 0,
            "state_1": 0,
            "f_list": [],
            "last": [-1],
            "early": [1e15],
        }
    if bid not in banks:
        banks.add(bid)
    uid_bid_dict[uid][bid]["cnt"]+=1
    if item[14] == 1:
        uid_bid_dict[uid][bid]['state_1'] += 1
    uid_bid_dict[uid][bid]['f_list'].append(item[3:])

    if ctime > uid_bid_dict[uid][bid]["last"][0]:
        uid_bid_dict[uid][bid]["last"] = [ctime, item[5]]
    if ctime < uid_bid_dict[uid][bid]["early"][0]:
        uid_bid_dict[uid][bid]["early"] = [ctime, item[5]]
    if uid not in uid_last_time:
        uid_last_time[uid] = ctime
    if ctime > uid_last_time[uid]:
        uid_last_time[uid] = ctime


uid_bid_res = {}
for uid, x in tqdm(uid_bid_dict.items()):
    uid_bid_res[uid] = {}
    for bid, v in x.items():
        f_list = v['f_list']
        uid_bid_res[uid][bid] = {
            # "state_1": v["state_1"],
            "cnt": v["cnt"],
            "change": v["last"][1] - v["early"][1],
            "sum_features": do_helper(v["f_list"], "sum"),
            "mean_features": do_helper(v["f_list"], "mean"),
            "max_features": do_helper(v["f_list"], "max"),
            "min_features": do_helper(v["f_list"], "min"),
            "std_features": do_helper(v["f_list"], "std"),
            "25median_features": do_helper(v["f_list"], "25"),
            "50median_features": do_helper(v["f_list"], "50"),
            "75median_features": do_helper(v["f_list"], "75"),
        }

for uid in users:
    if uid in uid_bid_res:
        for bid in sorted(banks):
            if bid not in uid_bid_res[uid]:
                outputs[uid] += "0," * (15*8+2)
            else:
                for key in sorted(uid_bid_res[uid][bid].keys()):
                    outputs[uid] += map_list2str(uid_bid_res[uid][bid][key])
    else:
        print("miss", uid)
        assert "missing uid:{} in uid_bid_res".format(uid)

section 3


100%|██████████| 53174/53174 [00:43<00:00, 1235.15it/s]


In [5]:
####section 4
print("section 4")
for t in [86400*30, 86400*90, 86400*365]:
    uid_dict = {}
    for item in all_data:
        [uid, ctime, bid] = item[0:3]
        ctime = int(ctime)
        if ctime < uid_last_time[uid] - t:
            continue
        if uid not in uid_dict:
            uid_dict[uid] = {
                "bank": set(),
                "cnt": 0,
                "f_list": [],
            }
        uid_dict[uid]["bank"].add(bid)
        uid_dict[uid]["cnt"]+=1
        uid_dict[uid]['f_list'].append(item[3:])

    uid_res = {}    
    for uid, v in tqdm(uid_dict.items()):
        f_list = v['f_list']
        uid_res[uid] = {
            "bank_size": len(v["bank"]),
            "cnt": v["cnt"],
            "sum_features": do_helper(v["f_list"], "sum"),
            "mean_features": do_helper(v["f_list"], "mean"),
            "max_features": do_helper(v["f_list"], "max"),
            "min_features": do_helper(v["f_list"], "min"),
            "std_features": do_helper(v["f_list"], "std"),
            "25median_features": do_helper(v["f_list"], "25"),
            "50median_features": do_helper(v["f_list"], "50"),
            "75median_features": do_helper(v["f_list"], "75"),
        }

    for uid in users:
        if uid in uid_res:
            for key in sorted(uid_res[uid].keys()):
                outputs[uid] += map_list2str(uid_res[uid][key])
        else:
            outputs[uid] += "0," * (15*8+2) 
            print("miss", uid)


section 4


100%|██████████| 53174/53174 [00:26<00:00, 1987.93it/s]


855


In [11]:
headers = "uid," + ','.join(map(lambda x: "x_"+str(x), range(1, 855)))

fout = open("./outputs_0720_test.csv", 'w')
fout.write(headers + '\n')
for uid in users:
    msg = uid_labels[uid] + outputs[uid]
    msg = msg.strip(',')
    fout.write(msg + '\n')

print(len(msg.split(',')))

855


In [12]:
import pandas as pd
b0 = pd.read_csv("./outputs1_0720_test.csv")
b0

,uid,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_845,x_846,x_847,x_848,x_849,x_850,x_851,x_852,x_853,x_854
0,3150,0.000000,0.000000,0.000000,17.728274,15.877126,0.00,0.000000,0.0,0.000000,...,27.0,236.604990,58.008847,0.000000,0.0,507.332845,-29.604743,-224.424184,148.991400,519.444849
1,6965,0.000000,6.849967,19.971271,19.956542,16.964090,4.25,18.437633,0.0,15.288139,...,36.0,39.939378,0.000000,30.576278,0.0,39.942542,-13.699934,39.939378,56.815722,39.942542
2,1265,19.599894,17.563325,19.748127,19.708086,17.174520,0.00,18.291582,0.0,0.000000,...,76.0,1011.943520,0.000000,263.440629,0.0,504.453942,183.184108,75.466759,1458.568257,1058.409216
3,6360,19.070165,16.972520,19.359079,19.687886,17.457298,0.00,0.000000,0.0,0.000000,...,192.0,1203.352137,0.000000,779.216182,0.0,1257.018504,143.265297,-429.471723,2125.833616,1694.791726
4,2583,18.106398,16.608570,19.460445,18.424931,7.795901,1.00,18.172761,0.0,0.000000,...,42.0,393.137051,0.000000,155.478508,0.0,245.584312,-1.145909,47.907417,547.469650,419.910567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53168,55348,0.000000,0.000000,20.846739,19.293851,17.947665,0.00,0.000000,0.0,0.000000,...,2.0,173.430998,-54.057012,30.196673,0.0,182.102715,3.794839,19.958204,261.479522,439.340478
53169,55349,21.776117,0.000000,21.763030,21.794800,19.479653,0.00,15.784679,0.0,17.663150,...,0.0,15.784679,0.000000,17.663150,0.0,21.069883,21.776117,-5.991438,55.223946,21.763030
53170,55350,20.503138,20.544453,21.827569,20.645692,19.254744,0.00,20.458794,0.0,0.000000,...,0.0,988.588609,0.000000,309.580297,0.0,1018.468800,52.064239,-20.132691,1350.233145,1052.770494
53171,55351,0.000000,0.000000,0.000000,19.744689,17.511798,0.00,0.000000,0.0,0.000000,...,0.0,784.032035,0.000000,0.000000,0.0,435.954288,252.976502,-107.275321,1037.008537,895.293521


In [13]:
a0 = pd.read_csv("/mnt/c/Users/cymle/Desktop/data/user_info_onehot.csv")
a0

,uid,label,sex_0,sex_1,sex_2,career_0,career_1,career_2,career_3,career_4,...,marital_1,marital_2,marital_3,marital_4,marital_5,account_0,account_1,account_2,account_3,account_4
0,1,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
1,2,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
2,3,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
3,4,1,0,1,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
4,5,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55591,55592,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
55592,55593,1,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,1
55593,55594,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
55594,55595,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1


In [14]:
b1 = pd.merge(a0, b0, how='inner', on=['uid'])

In [16]:
b1.to_csv("/mnt/c/Users/cymle/Desktop/data/output_no_norm.csv", index=False)
b1

,uid,label,sex_0,sex_1,sex_2,career_0,career_1,career_2,career_3,career_4,...,x_845,x_846,x_847,x_848,x_849,x_850,x_851,x_852,x_853,x_854
0,2,0,0,1,0,0,0,1,0,0,...,53.0,335.695735,0.000000,0.000000,0.000000,142.637152,9.347233,-7.649129,345.042968,387.655252
1,3,0,0,1,0,0,0,0,0,1,...,4.0,37.162823,0.000000,0.000000,0.000000,0.000000,-0.426806,0.429281,36.736017,36.723666
2,4,1,0,1,0,0,0,0,0,1,...,0.0,230.765089,0.000000,0.000000,0.000000,243.569409,66.826995,2.760489,297.592084,477.742636
3,5,0,0,1,0,0,0,1,0,0,...,0.0,35.914512,0.000000,0.000000,0.000000,39.942542,4.278933,-5.443941,40.193445,41.328836
4,6,1,0,1,0,0,0,1,0,0,...,53.0,214.111454,0.000000,0.000000,82.792589,219.744957,9.508023,-63.647776,223.619477,246.194967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53168,55592,0,0,1,0,0,0,1,0,0,...,0.0,96.275173,17.530205,48.008470,0.000000,98.858418,-23.111481,36.529561,103.641957,100.244712
53169,55593,1,0,1,0,0,0,0,0,1,...,0.0,377.436802,0.000000,16.891939,0.000000,258.008629,95.240441,-22.316083,489.569182,609.424985
53170,55594,0,0,1,0,0,0,1,0,0,...,12.0,407.869112,116.322561,0.000000,0.000000,214.753480,4.596329,-7.295301,296.142880,431.289730
53171,55595,0,0,1,0,0,0,1,0,0,...,2.0,463.886648,60.383863,16.489600,0.000000,478.394615,-13.295941,119.081162,406.696444,487.405526
